In [1]:
import pandas as pd
import numpy as np

def import_data(filename):
    df_ = pd.DataFrame()
    d = np.loadtxt(filename)
    df = pd.DataFrame(d, columns=['YYYYMMDD', 'HH', 'Zonal price', 'System load', 'Zonal load', 'Day of the week'])
    df_['price'] = df['Zonal price']
    return df_

filename = '/Users/sebastiansuwada/Desktop/Predictive Analytics/Naive_probabilistic_forecasts_List_3/GEFCOM.txt'
df = import_data(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25968 entries, 0 to 25967
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   25968 non-null  float64
dtypes: float64(1)
memory usage: 203.0 KB


Obtain Naive forecast for prepare of Point Forcast

In [4]:
def CombinationNaiveForecast(df_1, shift1, shift2):

    # P_w = (P_(w-1)+P_(w-3))/2
    naiv1 = pd.DataFrame()
    naiv1= df_1['price'].shift(shift1)
    naiv2 = pd.DataFrame()
    naiv2= df_1['price'].shift(shift2)
    
    naiv1 = naiv1.dropna()
    naiv2 = naiv2.dropna()
    naiv1.reset_index(drop=True, inplace=True)
    naiv2.reset_index(drop=True, inplace=True)

    min_size = min(len(naiv1), len(naiv2), len(df_1))
    naiv1 = naiv1.tail(min_size)
    naiv2 = naiv2.tail(min_size)
    df_sized = df_1.tail(min_size).copy()

    dfs = [naiv1, naiv2, df_sized]
    for df in dfs:
        df.reset_index(drop=True, inplace=True)

    naivSum = pd.DataFrame()
    naivSum['price'] = (naiv1 + naiv2)/2

    return naivSum, df_sized

shift1 = 24
shift2 = 168
df_sized, naivSum = CombinationNaiveForecast(df, shift1, shift2)
print(naivSum)


        price
0       49.22
1       42.69
2       40.33
3       39.81
4       39.38
...       ...
25795  113.92
25796  107.26
25797   89.02
25798   85.40
25799   86.13

[25800 rows x 1 columns]


Now obtained point forecast can be used for Quantile Regression the dataFrame called naivSum now is our new data that we are going to work on

In [42]:
def RegressionModel(df, perc, coverage):
    errors = []
    T = 364

    for h in range(24):
        
        # Create list p_hour that includes 364 values for specyfic hours of point forecast defined from df
        p_hour = df.loc[h::24, 'price'].values
        cal_data = p_hour[:T]
        day = p_hour[T:]
        hits = []


        Y = cal_data[1:]  # Y is the next day's prices
        X1 = cal_data[:-1]  # X1 is today's prices

        # Create a design matrix (X) with only X1
        X = X1.reshape(-1, 1)

        # Estimate the beta
        beta = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, Y))

        # Create list "forecasts" what consists all of values for specyfic hours with window defined as T 
        # Created based on point forecast
        forecasts = []
        for x in day:
            X_fut = np.array([x]).reshape(-1, 1)
            forecast = np.dot(X_fut, beta)
            forecasts.append(forecast[0])

        print(forecasts)
        print(cal_data)

        # Calculate the error
        err = real[day] - forecast

        for day in range(0, len(cal_data)):

            # Condition for choosing the error formula
            if forecast < real:
                err = err * (1 - perc)
            else:
                err = err * perc

            print(err)

            quants = np.quantile(err, [(1-coverage)/2, (1+coverage)/2])
            print(quants)
            low_PI = forecast - abs(quants[0])
            up_PI = forecast + abs(quants[1])

            print(up_PI)
            print(low_PI)
            print(real)

            if real < up_PI and real > low_PI:
                hits.append(1)
            else:
                hits.append(0)

    print(len(hits))

            # Jak mamy error jak różnicę to zapisujemy


# Split data into half

naivSumTrain = naivSum['price'][:int(len(naivSum)/2)]
naivSumTrain.info()

RegressionModel(naivSum, 0.95, 0.5)


<class 'pandas.core.series.Series'>
RangeIndex: 12900 entries, 0 to 12899
Series name: price
Non-Null Count  Dtype  
--------------  -----  
12900 non-null  float64
dtypes: float64(1)
memory usage: 100.9 KB
[28.751509677146224, 27.55927778169225, 29.717118980902335, 30.101392071172622, 32.34791167582969, 32.43659008127669, 31.096560843411062, 45.669378805199706, 67.68132966837472, 61.069861884493584, 34.752081779059196, 34.11162662860871, 39.7279256402514, 32.9785136701194, 34.92943858995318, 41.018689097313136, 30.308308350548934, 29.90432894795709, 28.43620868000137, 29.805797386349322, 26.347339573916724, 28.455914992322924, 29.273726953667385, 28.337677118393607, 28.79092230178933, 24.36685518560062, 24.33729571711829, 25.963066483646436, 29.96344788492175, 29.145635923577288, 25.77585651659168, 26.938528943563323, 27.490305688566814, 28.672684427860013, 27.194711003743517, 37.98391699979393, 46.23100870636398, 43.974635945546126, 27.726781436425455, 25.697031267305466, 25.92365385

IndexError: arrays used as indices must be of integer (or boolean) type